### Spark 데이터 처리 프레임워크

|     |비 실시간 처리|실시간 처리|
|---|---|---|
|정형 데이터|Dataframe|Structured Streaming|
|비정형 데이터|RDD|Dstream|


### 빅데이터의 현실적 정의
- 정의
    - 현재의 데이터베이스 시스템 아키텍처와 관계형 DBMS를 사용해서는 처리 및 관리가 쉽지 않은 데이터
    - 현재의 관계형 데이터베이스 시스템 아키텍처
        - 단일 또는 분산 서버가 마치 하나의 서버에 데이터가 저장된 것처럼
          (투명성 (transparency)) 저장 관리
        - 데이터의 무결성을 유지하면서 갱신 관리 (ACID)
        

### Stream Processing Overview

- Stream data를 아주 작은 RDD로 나누어서, RDD처럼 처리
-> DStream(discretized Streams)

### DStream Sources

1. socketTextStream
    - TCP/IP 소켓에서 데이터를 바로 수신
    - hostname과 port number를 지정
    - 테스트용으로 사용
    
2. textFileStream
    - 데이터 입력 폴더에서, 새로이 만들어지는 파일의 데이터를 읽어들임
    - 기존에 데이터가 파일로 있을 경우나 새로 만들어지는 데이터들이 파일로 저       장될 경우, 이렇게 생성되는 파일들을 읽어서 스트리밍 데이터를 처리.

3. Kafka

### socketTextStream을 위한 컴퓨터 환경 구성
- 윈도우에서 2개의 터미널을 연결.
    - 첫번째 터미널: pyspark 프로그램 실행
        - Anaconda의 Jupyter Notebook에서 프로그래밍 작업 수행

- 두번째 터미널: 스트리밍 데이터 생성
    - 터미널에서 송신 포트를 설정하고, 데이터 송신 준비를 함
      (anaconda의 환경설정 (Environment)에서 Open Terminal)을 선택하여 새로운 터미널을 엶

### socketTextStream을 위한 Spark Streaming Context 구성

- import libraries

        import pyspark
        from pyspark import SparkConf, SparkContext
        from pyspark.streaming import StreamingContext

- Master 지정
    - Localhost에 여러 개의 thread를 실행하여 그중 하나를 master로 지정 이때 서버와 클라이언트 사이의 데이터 통신이 필요하므로, master local(2) 등의 임의의 숫자를 입력하여 local server를 실행
    - 그냥 local로 쓰면, 1개의 thread만 실행됨을 의미하며, 따라서 스트리밍 데이터를 방생시키는 thread와 통신이 이루어지지 않음.
    
 sc = pyspark.SparkContext(master="local[2]")
 
 - 대기시간을 지정: Seconds()
 
 ssc = StreamingContext(sc,5) -> 5초 단위
 
 ### SparkStreaming에서 데이터 수신
 - socketTextStream:
     - 통신 socket으로 데이터를 수신, 주로 테스트 용도로 사용
     - port number는 nc(ncat)에서 지정한 port number와 동일해야 함
         -> 수신 데이터는 DStream (Discretized Stream)으로 변환됨.
         
 lines = ssc.socketTextStream("localhost",9999)
 
 ### DStream 데이터 변환 처리
 - RDD의 변환 함수들과 동일한 변환 함수들을 사용해서 Dstream 데이터를 변환 처리 함
 
         import re
         words = lines.flatMap(lambda line: re.split("\W+", line))

         words = words.filter(lambda word: len(word)>0) \
         .map(lambda w: w.lower()) \
         .map(lambda w: (w, 1)) \
         .reduceByKey(lambda v1, v2: v1 + v2) \
         .map(lambda r: (r[1],r[0])) \
         .transform(lambda rdd: rdd.sortByKey()) \
         .map(lambda r: (r[1],r[0]))

         words.pprint()
         
### 스트리밍 작업 시작과 실행
- Spark Streaming 데이터 처리 실행
    - ssc.start()
    - ssc.waitTermination 또는
      ssc.waitTerminatingOrTimeout(180) : Timeout 시간이 180 seconds인 경우

In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
import pyspark

In [4]:
from pyspark import SparkConf, SparkContext

In [5]:
from pyspark.streaming import StreamingContext

In [6]:
sc = pyspark.SparkContext(master="local[2]")

In [7]:
ssc = StreamingContext(sc,10)

In [8]:
lines = ssc.socketTextStream("localhost",9999)

In [9]:
import re

In [10]:
words = lines.flatMap(lambda line: re.split("\W+", line))

In [11]:
words = words.filter(lambda word: len(word)>0)

In [12]:
wordsLower = words.map(lambda w: w.lower())

In [13]:
wordAndOne = wordsLower.map(lambda w: (w,1))

In [14]:
wordCount = wordAndOne.reduceByKey(lambda v1, v2: v1 + v2)

### Tranformations on DStreams

- map(),flatMap(),filter(),union(),reduceByKey(),join() ...
- sortBy(),sortByKey()는 없음

- 해결방법
    - transform()을 활용:
      Returns a new DStream by applying a RDD-to-RDD function to every RDD of the source DStream. This can be used to do arbitrary RDD operations on the DStream
    - Example:
      .transform(lambda rdd: rdd.sortBy(lambda x: x[1],False))

In [15]:
sortedWC = wordCount.transform(lambda rdd:rdd.sortBy(lambda x:x[1],False))

In [16]:
sortedWC.pprint()

In [17]:
ssc.start()

In [18]:
ssc.awaitTermination()

1. netcat 설치
2. 환경변수 설정
3. cmd 창에서 nc.exe -lvp 9999 입력할 것!!